## Public Use Microdata Sample (PUMS) - Survey Data

In [165]:
import numpy as np
import pandas as pd
import os

# Added version check for recent scikit-learn 0.18 checks
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

%matplotlib inline

In [166]:
data=pd.read_csv(os.path.join("csv_pus",'psam_pusa.csv'))

In [167]:
print(f"Number of (rows,columns): {data.shape}")

Number of (rows,columns): (4691835, 286)


In [168]:
data.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2013000000084,6,1,2600,3,1,1061971,13,19,...,14,1,1,27,24,13,14,14,13,27
1,P,2013000000154,6,1,2500,3,1,1061971,11,55,...,19,13,13,19,11,4,3,4,12,3
2,P,2013000000154,6,2,2500,3,1,1061971,13,56,...,22,13,15,23,12,4,4,5,12,4
3,P,2013000000154,6,3,2500,3,1,1061971,30,21,...,59,29,28,67,37,10,10,10,29,10
4,P,2013000000154,6,4,2500,3,1,1061971,15,21,...,28,12,14,30,13,4,4,4,17,6


In [63]:
data.columns

Index(['RT', 'SERIALNO', 'DIVISION', 'SPORDER', 'PUMA', 'REGION', 'ST',
       'ADJINC', 'PWGTP', 'AGEP',
       ...
       'PWGTP71', 'PWGTP72', 'PWGTP73', 'PWGTP74', 'PWGTP75', 'PWGTP76',
       'PWGTP77', 'PWGTP78', 'PWGTP79', 'PWGTP80'],
      dtype='object', length=286)

## Filter columns

In [5]:
to_collect=['AGEP','REGION','DIVISION','ST','ADJINC','COW','DEAR','MAR',
            'DDRS','DEAR','DOUT','DPHY','DRAT','INTP','SEX','RAC1P','POVPIP','PINCP']
label = ["HINS4"]

In [182]:
X = data.loc[:20000, to_collect]
y = data.loc[:20000, label]

In [170]:
X.shape

(20001, 18)

In [171]:
print(X.shape)

(20001, 18)


## Missing Values

In [172]:
missing_values = ["n/a", "na", "--"]
print(X.isnull().sum())

AGEP            0
REGION          0
DIVISION        0
ST              0
ADJINC          0
COW          9026
DEAR            0
MAR             0
DDRS         1016
DEAR            0
DOUT         3344
DPHY         1016
DRAT        19617
INTP         3344
SEX             0
RAC1P           0
POVPIP        888
PINCP        3344
dtype: int64


In [173]:
X.isin(missing_values).any()

AGEP        False
REGION      False
DIVISION    False
ST          False
ADJINC      False
COW         False
DEAR        False
MAR         False
DDRS        False
DEAR        False
DOUT        False
DPHY        False
DRAT        False
INTP        False
SEX         False
RAC1P       False
POVPIP      False
PINCP       False
dtype: bool

In [174]:
X["PINCP"].fillna(0.0, inplace=True)

In [175]:
X['PINCP'] = X['PINCP'].fillna(X['PINCP'].ix[0.0])

C:\Users\mchiv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [176]:
X = X.drop(['DRAT'], axis=1)


In [177]:
X = X.fillna(0.0)

In [178]:
y = y

In [179]:
X.head()

,AGEP,REGION,DIVISION,ST,ADJINC,COW,DEAR,MAR,DDRS,DEAR,DOUT,DPHY,INTP,SEX,RAC1P,POVPIP,PINCP
0,19,3,6,1,1061971,0.0,2,5,2.0,2,2.0,2.0,0.0,2,1,0.0,0.0
1,55,3,6,1,1061971,1.0,2,1,2.0,2,2.0,2.0,0.0,2,2,501.0,52000.0
2,56,3,6,1,1061971,6.0,2,1,2.0,2,2.0,2.0,0.0,1,2,501.0,99000.0
3,21,3,6,1,1061971,0.0,2,5,2.0,2,2.0,2.0,0.0,1,2,501.0,0.0
4,21,3,6,1,1061971,0.0,2,5,1.0,2,1.0,1.0,0.0,2,2,501.0,0.0


## Split the data - Train and Test
Splitting data into 70% training and 30% test data:

In [180]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [181]:
print(f"X_train:{X_train.shape}, X_test: {X_test.shape}")

X_train:(14000, 17), X_test: (6001, 17)


## Standardizing the features

In [131]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Computing Fairness Model

In [138]:
p_1 = X_train[X_train['PINCP']<=20000] ## 20,000k
p_0 = X_train[X_train['PINCP']>20000]

In [139]:
def compute_t(fairness):
    return (1/fairness) - 1


In [162]:
fairness = [0.2,0.4,0.6,0.8]
lambdas = [[5,5], [100,100], [1000,1000]]


In [163]:
## Fairness matrix
# protected attribute - income
def get_number(p_set, y, yvalue):
    row_index = p_set.index
    result = y.loc[row_index]
    return float(result[result==yvalue].sum())/y[y==yvalue].sum()

prior_matrix = [get_number(p_1,y_train,1),get_number(p_0,y_train,1)]


## Training Logistic Regression Model

In [164]:
from sklearn.linear_model import LogisticRegression

for f in fairness:
    t = compute_t(f)
    print(f"For Fairness,{f} => {t}")
    prior1 = prior_matrix[0] - (1/(1+t))
    prior2 = prior_matrix[1] - (t/(1+t))
    for (l1,l2) in lambdas:
        c = (l1*prior1 + l2*prior2)
        clf = LogisticRegression(C=c)
        clf.fit(X_train, y_train)
        print('C:', c)
        print('Coefficient of each feature:', clf.coef_)
        print('Training accuracy:', clf.score(X_train, y_train))
        print('Test accuracy:', clf.score(X_test, y_test))
        print('')

For Fairness,0.2 => 4.0
C: 105.0
Coefficient of each feature: [[ 1.32470361e-02 -3.38997702e-01 -5.18667292e-01  0.00000000e+00
  -4.73261619e-06 -7.35999360e-02  1.30430769e+00  1.71113924e-01
  -7.06512144e-02  1.10520834e+00  7.88841882e-01  1.87989862e+00
  -3.61075888e-04  2.75745336e-01 -2.40222371e-01  2.89531906e-03
   1.01885935e-04]]
Training accuracy: 0.9
Test accuracy: 0.9016393442622951

C: 2100.0
Coefficient of each feature: [[ 3.97662120e-03 -4.48368998e-01 -2.41554319e-01 -1.77172889e+00
  -1.44712155e-06 -6.15664302e-02  1.04906846e+00  7.12640656e-02
   2.65171925e-01  9.50982675e-01  8.14780308e-01  1.46563042e+00
  -3.52480229e-04  2.44958831e-01 -2.34017122e-01  2.77831716e-03
   9.56663343e-05]]
Training accuracy: 0.9071428571428571
Test accuracy: 0.9016393442622951

C: 21000.0
Coefficient of each feature: [[ 2.93407174e-03 -4.80389322e-01 -2.47815271e-01 -1.53094419e+00
  -1.57910507e-06 -5.93395187e-02  7.26576740e-01  5.64317647e-02
   3.34409620e-01  1.2630764

C:\Users\mchiv\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
